In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from library.Benchmark2 import *
import time

In [3]:
#bulk 100GB, overwrite 100GB
def test_data_disk_bulk_and_overwrite_with_compression(test_dir):
    rocks_db_dir = "/home/fxie/rocksdb"
    print(f"test dir {test_dir}")
    benchmark2 = Benchmark2(rocks_db_dir = rocks_db_dir, test_dir = test_dir)
    benchmark2.para_map["COMPRESSION_TYPE"] = "none"
    benchmark2.para_map["CACHE_SIZE"] = "104857600"
    benchmark2.para_map["NUM_KEYS"] = "250000000"
    benchmark2.bulkload()
        
    #reduce kv num
    #benchmark2.para_map["NUM_KEYS"] = "25000000"
    benchmark2.overwrite()
        
    benchmark2.readrandom()
        
    benchmark2.clean_up()   

In [4]:
test_data_disk_bulk_and_overwrite_with_compression("/read_cache")

test dir /read_cache
Setup a new test root. Test root dir /read_cache/test-980980134
running bulk load
 KEY_SIZE=20 VALUE_SIZE=400 DB_DIR=/read_cache/test-980980134/db OUTPUT_DIR=/read_cache/test-980980134/output WAL_DIR=/read_cache/test-980980134/wal TEMP=/read_cache/test-980980134/tmp COMPRESSION_TYPE=none CACHE_SIZE=104857600 NUM_KEYS=250000000 /home/fxie/rocksdb/tools/benchmark.sh bulkload
iostat monitor thread : iostat-bulkload start. timestamp : 2019-07-31 23:04:25.753128
iostat monitor thread : iostat-bulkload end. timestamp : 2019-07-31 23:31:22.866896
===== Benchmark =====
Start bulkload at Wed Jul 31 23:04:25 UTC 2019
Bulk loading 250000000 random keys
./db_bench --benchmarks=fillrandom --use_existing_db=0 --disable_auto_compactions=1 --sync=0 --db=/read_cache/test-980980134/db --wal_dir=/read_cache/test-980980134/wal --num=250000000 --disable_wal=true --num_levels=6 --key_size=20 --value_size=400 --block_size=8192 --cache_size=104857600 --cache_numshardbits=6 --compression_m

iostat monitor thread : iostat-overwrite end. timestamp : 2019-07-31 23:49:36.593495


KeyboardInterrupt: 